# Cell/particle Counting and scoring stained objects

This notebook demonstrates how to process plates associated to the paper ['Integration of biological data by kernels on graph nodes allows prediction of new genes involved in mitotic chromosome condensation.'](http://dx.doi.org/10.1091/mbc.E13-04-0221).
We use the [Cell/particle counting, and scoring the percentage of stained objects](http://cellprofiler.org/examples/#PercentPositive) pipeline.
We explore the features with [Principal Component Analysis](https://en.wikipedia.org/wiki/Principal_component_analysis).

### Import Packages

In [ ]:
# %tb

# Import Cell Profiler Dependencies
import cellprofiler
import cellprofiler.preferences as cpprefs
import cellprofiler.module as cpm
import cellprofiler.pipeline as cpp
cpprefs.set_headless()

# Inject Image module used to inject OMERO image planes into Cell Profiler Pipeline
from cellprofiler.modules.injectimage import InjectImage

# Import OMERO Python BlitzGateway
import omero
from omero.gateway import BlitzGateway

# Import Numpy
import numpy as np

# Import Python System Packages
import os
import tempfile
import pandas
import warnings

# Import Matplotlib
import matplotlib
import matplotlib.pyplot as plt

### Set Cell Output Directory

In [ ]:
new_output_directory = os.path.normcase(tempfile.mkdtemp())
cpprefs.set_default_output_directory(new_output_directory)

### OMERO Credentials

In [ ]:
HOST = 'outreach.openmicroscopy.org'
# To be modified
USERNAME = 'username'
PASSWORD = 'password'
PORT = 4064
client = omero.client(host=HOST, port=PORT)
client.createSession(USERNAME, PASSWORD)
conn = BlitzGateway(client_obj=client)

### Fetch OMERO Plate that contains the Images to be analysed

In [ ]:
# To be modified
plate_id = 422
plate = conn.getObject("Plate", plate_id)
print 'Plate Name: ', plate.getName()

In [ ]:
from os.path import expanduser
home = expanduser("~")
pipeline = cpp.Pipeline()
pipeline.load(home+"/notebooks/CellProfiler/pipelines/ExamplePercentPositive.cppipe")
for module in pipeline.modules():
    print  module.module_name

### Run Cell Profiler Pipeline on a plate

In [ ]:
warnings.filterwarnings('ignore')

Nuclei = pandas.DataFrame()
Image = pandas.DataFrame()
image_ids = []
well_ids = []
files = []

for well in plate.listChildren():
    
    index = well.countWellSample()
    well_ids.append(well.getId())
    for index in xrange(0, index):
        image = well.getImage(index))
        image_ids.append(image.getId())
        pixels = image.getPrimaryPixels()
        size_c = image.getSizeC()
        pipeline_copy = pipeline.copy()
        for c in range(0, size_c):
        
            plane = pixels.getPlane(0, c, 0)
            image_name = image.getName()
            
            if c == 0:
                image_name = 'OrigBlue'
            if c == 1:
                image_name = 'OrigGreen'
            if c == 2:
                image_name = 'OrigRed'
            
            inject_image_module = InjectImage(image_name, plane)
            inject_image_module.set_module_num(1)
            pipeline_copy.add_module(inject_image_module)
            
        for i in [1, 2, 3, 4]:
            pipeline_copy.remove_module(pipeline_copy.modules()[size_c].module_num)
        
        m = pipeline_copy.run()
        
        #Results obtained as CSV from Cell Profiler
        path = new_output_directory + '/Nuclei.csv'
        f = pandas.read_csv(path, index_col=None, header=0)
        f.ImageNumber = image.getName()
        files.append(f)

Nuclei = pandas.concat(files, ignore_index=True)

### Display the result from CellProfiler

In [ ]:
from IPython.core.display import HTML
display(HTML(Nuclei.to_html()))

### Calculate statistics

In [ ]:
Nuclei.describe()

In [ ]:
matplotlib.rcParams['figure.figsize'] = (18.0, 16.0)
Nuclei.hist();

### Principal Component Analysis (PCA)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
nb_columns = len(Nuclei.columns)-1
data = Nuclei.iloc[:,1:nb_columns]
std = StandardScaler().fit_transform(data)
pca = PCA(n_components=nb_columns-1)
x = pca.fit(std).transform(std)

### Visualize PCA result

#### Table List

In [ ]:
column_names = list(Nuclei.columns.values)
ylabels = column_names[1:nb_columns]
xlabels = range(1, nb_columns)
matplotlib.rcParams['figure.figsize'] = (12.0, 8.0)
figure = plt.figure()

axis = figure.add_subplot(1, 1, 1)
image = axis.pcolor(pca.components_.transpose(), cmap=matplotlib.cm.seismic, edgecolors='k', linewidths=2)
axis.xaxis.set(ticks=np.arange(0.5, len(xlabels)), ticklabels=xlabels)
axis.yaxis.set(ticks=np.arange(0.5, len(ylabels)), ticklabels=ylabels)
axis.set_ylim(axis.get_ylim()[::-1])
axis.xaxis.tick_top()
plt.colorbar(image);

#### Explained variance ratio

In [ ]:
figure = plt.figure()
axis = figure.add_subplot(1, 1, 1)
plt.plot(pca.explained_variance_ratio_, marker='o', linewidth=2)
axis.xaxis.set(ticklabels=range(1, nb_columns))
plt.xlabel('PCA component')
plt.grid(True)
plt.title('Explained variance ratio');

### Save the ouput to OMERO as OMERO.table

In [ ]:
from omero.grid import DoubleColumn, ImageColumn, LongColumn, WellColumn
from omero.constants.namespaces import NSBULKANNOTATIONS

img_column = ImageColumn('Image', '', image_ids)
well_column = WellColumn('Well', '', well_ids)
cols = []
for col in Nuclei.columns:
    if Nuclei[col].dtype == 'int64':
        cols.append(LongColumn(col, '', Nuclei[col]))
    elif Nuclei[col].dtype == 'float64':
        cols.append(DoubleColumn(col, '', Nuclei[col])

data = [well_column] + [img_column] + cols

resources = conn.c.sf.sharedResources()
repository_id = resources.repositories().descriptions[0].getId().getValue()
table_name = "idr0002_cellprofiler"
table = resources.newTable(repository_id, table_name)
table.initialize(data)
table.addData(data)
table.close()

# Link the table to the plate
orig_file = table.getOriginalFile()
file_ann = FileAnnotationWrapper(conn)
file_ann.setNs(NSBULKANNOTATIONS)
file_ann._obj.file = OriginalFileI(orig_file.id.val, False)
file_ann.save()
plate.linkAnnotation(file_ann)

<br>
### Close the connection to the OMERO server

In [ ]:
conn.close()